In [3]:
import pymysql
import requests

下載分析pm2.5 opendata資料

In [4]:
url="https://data.moenv.gov.tw/api/v2/aqx_p_02?api_key=e8dd42e6-9b8b-43f8-991e-b3dee723a52d&limit=1000&sort=datacreationdate%20desc&format=JSON"
url

'https://data.moenv.gov.tw/api/v2/aqx_p_02?api_key=e8dd42e6-9b8b-43f8-991e-b3dee723a52d&limit=1000&sort=datacreationdate%20desc&format=JSON'

In [5]:
resp=requests.get(url,verify=False)
resp

c:\Users\user\OneDrive\Desktop\jango\mysql\pm25_mysql\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'data.moenv.gov.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


<Response [200]>

In [6]:
datas=resp.json()['records']
datas

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [7]:
len(datas)

1000

In [7]:
datas[0]

{'site': '員林',
 'county': '彰化縣',
 'pm25': '18',
 'datacreationdate': '2025-08-21 22:00',
 'itemunit': 'μg/m3'}

建立資料表 8/20 1:00:00

In [8]:
table_str=""" 
create table if not exists pm25(
id int auto_increment primary key,
site varchar(25),
county varchar(50),
pm25 int,
datacreationdate datetime,
itemunit varchar(20),
unique key site_time(site,datacreationdate)
)
"""
print(table_str)

 
create table if not exists pm25(
id int auto_increment primary key,
site varchar(25),
county varchar(50),
pm25 int,
datacreationdate datetime,
itemunit varchar(20),
unique key site_time(site,datacreationdate)
)



連結資料庫

In [8]:
conn=pymysql.connect(
host="127.0.0.1",
user="root",
password="",
port=3307,
database="demo"
)
cursor= conn.cursor()
cursor

In [10]:
cursor.execute(table_str)
conn.commit()

寫入一筆資料

In [ ]:
sqlstr="insert into pm25(site,county,pm25,datacreationdate,itemunit)\
      values('{}','{}',{},'{}','{}')"
data=list(datas[0].values())
print(data)
sqlstr.format(data[0],data[1],data[2],data[3],data[4])

['員林', '彰化縣', '18', '2025-08-21 22:00', 'μg/m3']


"insert into pm25(site,county,pm25,datacreationdate,itemunit)      values('員林','彰化縣',18,'2025-08-21 22:00','μg/m3')"

In [12]:
cursor.execute(sqlstr.format(data[0],data[1],data[2],data[3],data[4]))
conn.commit()

### 一次寫入 8/20 1:40:00
- 加上 ignore 8/20 2:00:00

In [11]:
sqlstr="insert ignore into pm25(site,county,pm25,datacreationdate,itemunit)\
      values(%s,%s,%s,%s,%s)"
values=[list(data.values()) for data in datas if list(data.values())[2]!=""]
#values
cursor.executemany(sqlstr,values)
conn.commit()

In [ ]:
conn.close()

In [7]:
import pandas as pd

In [9]:
cursor.execute("select * from pm25")
datas=cursor.fetchall()
datas

((1, '員林', '彰化縣', 16, datetime.datetime(2025, 8, 22, 1, 0), 'μg/m3'),
 (2, '大城', '彰化縣', 15, datetime.datetime(2025, 8, 22, 1, 0), 'μg/m3'),
 (3, '富貴角', '新北市', 7, datetime.datetime(2025, 8, 22, 1, 0), 'μg/m3'),
 (4, '麥寮', '雲林縣', 15, datetime.datetime(2025, 8, 22, 1, 0), 'μg/m3'),
 (5, '關山', '臺東縣', 4, datetime.datetime(2025, 8, 22, 1, 0), 'μg/m3'),
 (6, '馬公', '澎湖縣', 3, datetime.datetime(2025, 8, 22, 1, 0), 'μg/m3'),
 (7, '金門', '金門縣', 3, datetime.datetime(2025, 8, 22, 1, 0), 'μg/m3'),
 (8, '馬祖', '連江縣', 5, datetime.datetime(2025, 8, 22, 1, 0), 'μg/m3'),
 (9, '埔里', '南投縣', 5, datetime.datetime(2025, 8, 22, 1, 0), 'μg/m3'),
 (10, '復興', '高雄市', 4, datetime.datetime(2025, 8, 22, 1, 0), 'μg/m3'),
 (11, '永和', '新北市', 4, datetime.datetime(2025, 8, 22, 1, 0), 'μg/m3'),
 (12, '竹山', '南投縣', 12, datetime.datetime(2025, 8, 22, 1, 0), 'μg/m3'),
 (13, '中壢', '桃園市', 7, datetime.datetime(2025, 8, 22, 1, 0), 'μg/m3'),
 (14, '冬山', '宜蘭縣', 6, datetime.datetime(2025, 8, 22, 1, 0), 'μg/m3'),
 (15, '宜蘭', '宜蘭縣', 6, da

In [13]:
df=pd.DataFrame(datas,columns=["id","site","county","pm25","datetime","unit"])
df

,id,site,county,pm25,datetime,unit
0,1,員林,彰化縣,16,2025-08-22 01:00:00,μg/m3
1,2,大城,彰化縣,15,2025-08-22 01:00:00,μg/m3
2,3,富貴角,新北市,7,2025-08-22 01:00:00,μg/m3
3,4,麥寮,雲林縣,15,2025-08-22 01:00:00,μg/m3
4,5,關山,臺東縣,4,2025-08-22 01:00:00,μg/m3
...,...,...,...,...,...,...
478,479,新莊,新北市,10,2025-08-21 19:00:00,μg/m3
479,480,板橋,新北市,10,2025-08-21 19:00:00,μg/m3
480,481,土城,新北市,9,2025-08-21 19:00:00,μg/m3
481,482,新店,新北市,7,2025-08-21 19:00:00,μg/m3


In [15]:
df.groupby("county").get_group("新北市").drop_duplicates()

,id,site,county,pm25,datetime,unit
2,3,富貴角,新北市,7,2025-08-22 01:00:00,μg/m3
10,11,永和,新北市,4,2025-08-22 01:00:00,μg/m3
61,62,淡水,新北市,10,2025-08-22 01:00:00,μg/m3
62,63,林口,新北市,12,2025-08-22 01:00:00,μg/m3
63,64,菜寮,新北市,7,2025-08-22 01:00:00,μg/m3
...,...,...,...,...,...,...
477,478,菜寮,新北市,12,2025-08-21 19:00:00,μg/m3
478,479,新莊,新北市,10,2025-08-21 19:00:00,μg/m3
479,480,板橋,新北市,10,2025-08-21 19:00:00,μg/m3
480,481,土城,新北市,9,2025-08-21 19:00:00,μg/m3


In [16]:
len(df)

483

In [18]:
df[["site","datetime"]].drop_duplicates()

,site,datetime
0,員林,2025-08-22 01:00:00
1,大城,2025-08-22 01:00:00
2,富貴角,2025-08-22 01:00:00
3,麥寮,2025-08-22 01:00:00
4,關山,2025-08-22 01:00:00
...,...,...
478,新莊,2025-08-21 19:00:00
479,板橋,2025-08-21 19:00:00
480,土城,2025-08-21 19:00:00
481,新店,2025-08-21 19:00:00
